[Link](https://runawayhorse001.github.io/LearningApacheSpark/regression.html)

# Set up spark context and SparkSession


In [13]:
try:
    sc.stop
except:
    pass

from pyspark import SparkContext , SparkConf
from pyspark.sql import SparkSession
sc = SparkContext.getOrCreate()
spark = SparkSession(sparkContext = sc)

#  Load dataset


In [14]:
df = spark.read.format('com.databricks.spark.csv').\
                       options(header='true', \
                       inferschema='true').\
            load("../data/Advertising.csv",header=True);
df.show(5,True)
df.printSchema()
df.describe().show()


+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)

+-------+-----------------+------------------+------------------+------------------+
|summary|               TV|             Radio|         Newspaper|             Sales|
+-------+-----------------+------------------+------------------+------------------+
|  count|              200|               200|               200|               200|
|   mean|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|              0.7|             

#  Convert the data to dense vector (features and label)

In [15]:
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

# I provide two ways to build the features and labels

# method 1 (good for small feature):
#def transData(row):
#    return Row(label=row["Sales"],
#               features=Vectors.dense([row["TV"],
#                                       row["Radio"],
#                                       row["Newspaper"]]))

# Method 2 (good for large features):
def transData(data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])

#  Transform the dataset to DataFrame


In [16]:
transformed= transData(df)
transformed.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[230.1,37.8,69.2]| 22.1|
| [44.5,39.3,45.1]| 10.4|
| [17.2,45.9,69.3]|  9.3|
|[151.5,41.3,58.5]| 18.5|
|[180.8,10.8,58.4]| 12.9|
+-----------------+-----+
only showing top 5 rows



#  Deal With Categorical Variables


In [17]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.

featureIndexer = VectorIndexer(inputCol="features", \
                               outputCol="indexedFeatures",\
                               maxCategories=4).fit(transformed)

data = featureIndexer.transform(transformed)
data.show(5,True)


+-----------------+-----+-----------------+
|         features|label|  indexedFeatures|
+-----------------+-----+-----------------+
|[230.1,37.8,69.2]| 22.1|[230.1,37.8,69.2]|
| [44.5,39.3,45.1]| 10.4| [44.5,39.3,45.1]|
| [17.2,45.9,69.3]|  9.3| [17.2,45.9,69.3]|
|[151.5,41.3,58.5]| 18.5|[151.5,41.3,58.5]|
|[180.8,10.8,58.4]| 12.9|[180.8,10.8,58.4]|
+-----------------+-----+-----------------+
only showing top 5 rows



# Split the data into training and test sets (40% held out for testing)

In [18]:
# Split the data into training and test sets (40% held out for testing)
(trainingData, testData) = transformed.randomSplit([0.6, 0.4])
trainingData.show(5)
testData.show(5)

+---------------+-----+
|       features|label|
+---------------+-----+
| [5.4,29.9,9.4]|  5.3|
|[7.3,28.1,41.4]|  5.5|
|[7.8,38.9,50.6]|  6.6|
| [8.4,27.2,2.1]|  5.7|
|  [8.6,2.1,1.0]|  4.8|
+---------------+-----+
only showing top 5 rows

+----------------+-----+
|        features|label|
+----------------+-----+
|  [0.7,39.6,8.7]|  1.6|
|  [4.1,11.6,5.7]|  3.2|
| [8.7,48.9,75.0]|  7.2|
|[11.7,36.9,45.2]|  7.3|
| [13.1,0.4,25.6]|  5.3|
+----------------+-----+
only showing top 5 rows



# LinearRegression

In [19]:
# Import LinearRegression class
from pyspark.ml.regression import LinearRegression
# Define LinearRegression algorithm
lr = LinearRegression()
# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, lr])
model = pipeline.fit(trainingData)



# Summary of the Model
Spark has a poor summary function for data and model. I wrote a summary function which has similar format as R output for the linear regression in PySpark.

In [20]:
def modelsummary(model):
    import numpy as np
    print ("Note: the last rows are the information for Intercept")
    print ("##","-------------------------------------------------")
    print ("##","  Estimate   |   Std.Error | t Values  |  P-value")
    coef = np.append(list(model.coefficients),model.intercept)
    Summary=model.summary

    for i in range(len(Summary.pValues)):
        print ("##",'{:10.6f}'.format(coef[i]),\
        '{:10.6f}'.format(Summary.coefficientStandardErrors[i]),\
        '{:8.3f}'.format(Summary.tValues[i]),\
        '{:10.6f}'.format(Summary.pValues[i]))

    print ("##",'---')
    print ("##","Mean squared error: % .6f" \
           % Summary.meanSquaredError, ", RMSE: % .6f" \
           % Summary.rootMeanSquaredError )
    print ("##","Multiple R-squared: %f" % Summary.r2, ", \
            Total iterations: %i"% Summary.totalIterations)
modelsummary(model.stages[-1])

Note: the last rows are the information for Intercept
## -------------------------------------------------
##   Estimate   |   Std.Error | t Values  |  P-value
##   0.045114   0.001540   29.299   0.000000
##   0.196999   0.009781   20.141   0.000000
##  -0.002401   0.006222   -0.386   0.700238
##   2.914105   0.351133    8.299   0.000000
## ---
## Mean squared error:  2.232753 , RMSE:  1.494240
## Multiple R-squared: 0.917570 ,             Total iterations: 1


#  Make predictions


In [21]:
# Make predictions.
predictions = model.transform(testData)
predictions

DataFrame[features: vector, label: double, indexedFeatures: vector, prediction: double]

In [22]:
predictions.select("features","label","indexedFeatures").show()

+----------------+-----+----------------+
|        features|label| indexedFeatures|
+----------------+-----+----------------+
|  [0.7,39.6,8.7]|  1.6|  [0.7,39.6,8.7]|
|  [4.1,11.6,5.7]|  3.2|  [4.1,11.6,5.7]|
| [8.7,48.9,75.0]|  7.2| [8.7,48.9,75.0]|
|[11.7,36.9,45.2]|  7.3|[11.7,36.9,45.2]|
| [13.1,0.4,25.6]|  5.3| [13.1,0.4,25.6]|
|[17.2,45.9,69.3]|  9.3|[17.2,45.9,69.3]|
|[19.6,20.1,17.0]|  7.6|[19.6,20.1,17.0]|
| [25.6,39.0,9.3]|  9.5| [25.6,39.0,9.3]|
|[36.9,38.6,65.6]| 10.8|[36.9,38.6,65.6]|
| [39.5,41.1,5.8]| 10.8| [39.5,41.1,5.8]|
|[43.1,26.7,35.1]| 10.1|[43.1,26.7,35.1]|
|[44.7,25.8,20.6]| 10.1|[44.7,25.8,20.6]|
| [53.5,2.0,21.4]|  8.1| [53.5,2.0,21.4]|
| [56.2,5.7,29.7]|  8.7| [56.2,5.7,29.7]|
|[57.5,32.8,23.5]| 11.8|[57.5,32.8,23.5]|
|[59.6,12.0,43.1]|  9.7|[59.6,12.0,43.1]|
|[62.3,12.6,18.3]|  9.7|[62.3,12.6,18.3]|
| [66.1,5.8,24.2]|  8.6| [66.1,5.8,24.2]|
|[70.6,16.0,40.8]| 10.5|[70.6,16.0,40.8]|
|[75.3,20.3,32.5]| 11.3|[75.3,20.3,32.5]|
+----------------+-----+----------

#  Evaluation

In [23]:
from pyspark.ml.evaluation import RegressionEvaluator
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="label",
                                predictionCol="prediction",
                                metricName="rmse")

rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 1.97036


In [24]:
y_true = predictions.select("label").toPandas()
y_pred = predictions.select("prediction").toPandas()

import sklearn.metrics
r2_score = sklearn.metrics.r2_score(y_true, y_pred)
print('r2_score: {0}'.format(r2_score))

r2_score: 0.8525568836200761
